In [1]:
import os

import numpy as np
import gymnasium as gym

import gym_env

In [2]:
# Construct the environment
env = gym.make("simple-15x15")
env.reset()

/Users/abizzle/anaconda3/envs/gym/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:181: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.deprecation(


({'agent': array([0, 0]), 'target': array([14, 14])}, {'distance': 28.0})

In [3]:
actions = np.arange(env.action_space.n, dtype=int)
start_loc = env.unwrapped.start_loc
target_loc = env.unwrapped.target_loc
maze = env.unwrapped.maze
size = maze.size
target_locs = [target_loc]

In [4]:
print(f"actions: {actions}, start loc: {start_loc}, target loc: {target_loc}, size: {size}")

actions: [0 1 2 3], start loc: [0 0], target loc: [14 14], size: 225


In [5]:
def row_col_to_index(row, col, len):
    """
    Converts (row,col) to an index in array
    """
    return row*len + col

def index_to_row_col(index, len):
    """
    Converts index back to (row,col)
    """
    return (index // len, index % len)

In [6]:
# Uncomment to test row_col_to_index and index_to_row_col functions
# for row in range(maze.shape[0]):
#     for col in range(maze.shape[1]):
#         index = row_col_to_index(row, col, maze.shape[0])
#         print((row,col), index, index_to_row_col(index, maze.shape[0]))

In [7]:
def get_blocked_states(maze):
    blocked_states = []

    for i in range(maze.shape[0]):
        for j in range(maze.shape[1]):
            if maze[i,j] == "1":
                blocked_states.append((i,j))
    
    return blocked_states

In [8]:
def create_transition_matrix_mapping(maze):
    blocked_states = get_blocked_states(maze)
    n = len(maze)  # Size of the maze (N)

    # Create a mapping from maze state indices to transition matrix indices
    mapping = {}
    matrix_idx = 0

    for i in range(n):
        for j in range(n):
            if (i, j) not in blocked_states:
                mapping[(i, j)] = matrix_idx
                matrix_idx += 1

    return mapping, len(mapping)

In [9]:
mapping, m = create_transition_matrix_mapping(maze)
reverse_mapping = {index: (i, j) for (i, j), index in mapping.items()}

In [10]:
# Get the transition matrix T N^2 x N^2
T = np.zeros(shape=(m, m))

# loop through the maze
for row in range(maze.shape[0]):
    for col in range(maze.shape[1]):
        # if we hit a barrier
        if maze[row,col] == '1':
            continue
            
        # at each location, we want to store the location, keep track of which new states we transition into, and how many states we transition into
        loc = np.array((row,col))
        idx_cur = mapping[row, col]

        # if we hit a goal
        if maze[row, col] == 'G':
            T[idx_cur, idx_cur] = 1
            continue

        new_states = []
        for action in actions:     # loop through actions
            env.unwrapped.agent_loc = loc                  # set new agent location based on where we are in maze
            obs, reward, term, _, _ = env.step(action)     # take action

            # if dont move because we hit a boundary, do nothing
            if (obs['agent'] == loc).all():
                continue
            new_states.append(obs['agent'])
        
        for new_state in new_states:
            idx_new =mapping[new_state[0], new_state[1]]
            T[idx_cur, idx_new] = 1/len(new_states)

In [11]:
# for i in range(len(T)):
#     print(sum(T[i]))
# print(T[-1])
assert np.all(np.isclose(np.sum(T, axis=1), 1.0)), "Not all rows sum to one."

# Old Method

In [13]:
"""
Split our T into T_nn & T_nt
T_nn -> transition probability between non-terminal states 
T_nt = P -> transition probability from non-terminal to terminal states
"""

# Make T_nn by excluding the rows and columns associated with the terminal state (also works if we have multiple)
# target_locs = [target_loc]
# terminal_indices = [mapping[loc[0], loc[1]] for loc in target_locs]

# T_nn = T.copy()

# for index in terminal_indices:
#     T_nn = np.delete(T_nn, index, axis=0)
#     T_nn = np.delete(T_nn, index, axis=1)

# # Make T_nt by selecting only the rows corresponding to the terminal states
# all_indices = set(range(T.shape[0]-1))
# nonterminal_indices = all_indices - set(terminal_indices)

# T_nt = np.zeros((len(T)-1, len(terminal_indices)))

# for i, index_term in enumerate(terminal_indices):
#     for index in nonterminal_indices:
#         T_nt[index, i] = T[index, index_term]

'\nSplit our T into T_nn & T_nt\nT_nn -> transition probability between non-terminal states \nT_nt = P -> transition probability from non-terminal to terminal states\n'

In [14]:
# print(T.shape)
# print(T_nt.shape)
# print(T_nn.shape)

In [15]:
# for i in range(len(T_nt)):
#     print(reverse_mapping[i], T_nt[i])

In [16]:
"""
Now we can use T_nn to solve for our DR (M)
"""
# _lambda = 1     # define lambda
# c = np.full(T_nn.shape[0], 1)     # define our cost to be 1 as our reward is negative 1

# # Make our diagonal matrix
# diag_matrix = np.diag(np.exp(c / _lambda))

# # Subtract from T_nn to get L
# L = diag_matrix - T_nn

# # Take the inverse to obtain the DR (M)
# M = np.linalg.inv(L)

'\nNow we can use T_nn to solve for our DR (M)\n'

In [17]:
# M.shape

In [18]:
"""
Now that we have M and P (T_nt), we can solve for exp_v
"""
# t = len(target_locs)
# r = np.full(t, 0)  # Create the vector r filled with 2

# # Calculate the right-hand side (RHS) of the equation
# exp_v = M @ T_nt @ np.exp(r / _lambda)

'\nNow that we have M and P (T_nt), we can solve for exp_v\n'

In [19]:
# exp_v

In [20]:
# we need to add back the terminal states
# holder = np.zeros(T.shape[0])

# for idx in terminal_indices:
#     holder[idx] = np.exp(r)
# for i, idx in enumerate(nonterminal_indices):
#     holder[idx] = exp_v[i]

In [21]:
# exp_v = np.copy(holder)

In [97]:
# solve for the optimal policy by choosing the next optimal state
# optimal_policy = np.zeros(len(exp_v))
# print(optimal_policy.shape, exp_v.shape)
# env.reset()

# for row in range(maze.shape[0]):
#     for col in range(maze.shape[1]):
#         # if we hit a barrier or terminal
#         if maze[row,col] == '1' or maze[row,col] == 'G':
#             continue

#         loc = np.array((row,col))
#         loc_index = row_col_to_index(row, col, maze.shape[0]-1)

#         max_action = 0
#         for action in actions:     # loop through actions
#             env.unwrapped.agent_loc = loc                  # set new agent location based on where we are in maze
#             obs, reward, term, _, _ = env.step(action)     # take action

#             # if dont move because we hit a boundary, do nothing
#             if (obs['agent'] == loc).all():
#                 continue

#             succ_state_index = row_col_to_index(obs['agent'][0], obs['agent'][1], maze.shape[0]-1)     # index of successor state
#             if exp_v[succ_state_index] > max_action: 
#                 optimal_policy[loc_index] = action
#                 max_action = exp_v[succ_state_index]


In [22]:
# print(exp_v)
# print(np.log(exp_v))

In [23]:
# v_maze = np.zeros_like(maze)
# for row in range(v_maze.shape[0]):
#     for col in range(v_maze.shape[1]):
#         if maze[row, col] == "1":
#             v_maze[row,col] = "BAR"
#             continue
#         v_maze[row,col] = round(np.log(exp_v[mapping[(row,col)]]), 2)

In [24]:
# Set formatting options
# print(v_maze)

# New method

In [26]:
print(maze)

[['S' '0' '1' '0' '0' '0' '1' '0' '0' '1' '0' '0' '0' '0' '0']
 ['0' '0' '0' '1' '0' '0' '0' '1' '0' '0' '1' '0' '0' '0' '0']
 ['0' '0' '0' '1' '0' '0' '0' '1' '0' '0' '1' '0' '0' '0' '0']
 ['0' '0' '1' '0' '0' '0' '0' '1' '0' '0' '1' '1' '0' '0' '0']
 ['0' '0' '1' '0' '0' '0' '0' '1' '1' '0' '0' '1' '0' '0' '0']
 ['0' '0' '1' '0' '0' '0' '0' '1' '0' '0' '0' '0' '1' '1' '1']
 ['0' '0' '0' '1' '0' '0' '0' '0' '0' '0' '0' '1' '1' '0' '0']
 ['0' '0' '0' '1' '1' '1' '1' '0' '0' '0' '0' '1' '0' '0' '0']
 ['0' '0' '0' '1' '0' '0' '0' '0' '1' '0' '0' '1' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0' '0' '1' '1' '0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '1' '1' '1' '0' '1' '1' '1' '1' '0' '0' '0']
 ['0' '0' '0' '0' '1' '0' '0' '0' '0' '0' '1' '0' '0' '0' '0']
 ['0' '0' '0' '0' '1' '0' '0' '0' '0' '0' '1' '0' '0' '1' '1']
 ['0' '0' '0' '0' '1' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '1' '0' '0' '0' '0' '0' '0' '0' '0' '0' 'G']]


In [27]:
terminals = np.diag(T) == 1

In [31]:
# Define cost and lambda and reward
# Calculate L
_lambda = 1     # define lambda
c = np.full(len(T), 1)     # define our cost to be 1 as our cost is -1
c[terminals] = 0
r = -c

In [32]:
L = np.diag(np.exp(c / _lambda)) - T

# Remove rows and columns corresponding to terminals
L = np.delete(L, terminals, axis=0)
L = np.delete(L, terminals, axis=1)

# Calculate the inverse of L
M = np.linalg.inv(L)

In [33]:
M.shape

(174, 174)

In [34]:
# Calculate P = T_{NT}
P = T[~terminals][:,terminals]

# Calculate expr
expr = np.exp(r[terminals] / _lambda)

# Initialize expv as zeros
expv = np.zeros(r.shape)

# Calculate expv_N for non-terminal states
expv_N = M @ P @ expr
expv[~terminals] = expv_N

# Calculate expv for terminal states
expv[terminals] = np.exp(r[terminals] / _lambda)

In [35]:
print(M.shape)
print(P.shape)
print(expr.shape)
print(expv.shape)
print(expv_N.shape)

(174, 174)
(174, 1)
(1,)
(175,)
(174,)


In [36]:
expv

array([1.90597507e-26, 1.76984309e-26, 2.63043881e-20, 7.15027401e-20,
       1.88075445e-19, 2.65430300e-17, 7.21514362e-17, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       8.59211169e-26, 7.71588955e-26, 3.90719118e-26, 3.68713964e-19,
       9.50981390e-19, 3.85842837e-18, 3.65712846e-16, 1.12606834e-15,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       6.04454786e-25, 6.96267035e-25, 1.35258040e-25, 1.98432128e-18,
       5.92492394e-18, 2.00256101e-17, 1.78411198e-15, 5.75622934e-15,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       4.14704724e-24, 6.75372839e-24, 4.73194575e-18, 9.88819550e-18,
       4.14615396e-17, 1.53522404e-16, 8.42721532e-15, 4.40309806e-14,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.64603462e-23,
       5.02322972e-23, 1.58373288e-17, 5.93378019e-17, 2.81481075e-16,
       1.19046433e-15, 3.44882398e-13, 8.94712082e-13, 0.00000000e+00,
      

In [37]:
v_maze = np.zeros_like(maze)
for row in range(v_maze.shape[0]):
    for col in range(v_maze.shape[1]):
        if maze[row, col] == "1":
            v_maze[row,col] = "BAR"
            continue
        v_maze[row,col] = round(np.log(expv[mapping[(row,col)]]), 2)

/var/folders/sf/4_xlxj4j2r1_sx5v07843wbw0000gn/T/ipykernel_54424/3308367700.py:7: RuntimeWarning: divide by zero encountered in log
  v_maze[row,col] = round(np.log(expv[mapping[(row,col)]]), 2)


In [43]:
v_maze.shape

(15, 15)

In [49]:
# Calculate G = T * expv
G = T.dot(expv)

# Calculate zg = (expv' / G)
zg = (expv.T / G).T

# Calculate pii = T * zg
pii = T * zg